In [1]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport

# Read  data

In [2]:
df = pd.read_csv('../data/train_mfcc.csv')
df.head()

,track id,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,power,joyful_activation,tension,sadness,mood,liked,disliked,age,gender,mother tongue
0,1,0.168761,0.084875,1250.061078,1383.643005,1891.738561,0.074272,-320.786858,109.852414,-13.965514,...,0,0,0,1,3,0,0,30,0,English
1,1,0.168761,0.084875,1250.061078,1383.643005,1891.738561,0.074272,-320.786858,109.852414,-13.965514,...,0,0,0,1,1,1,0,33,0,Russian
2,1,0.168761,0.084875,1250.061078,1383.643005,1891.738561,0.074272,-320.786858,109.852414,-13.965514,...,0,0,0,1,4,0,1,51,0,English
3,1,0.168761,0.084875,1250.061078,1383.643005,1891.738561,0.074272,-320.786858,109.852414,-13.965514,...,0,0,0,0,3,0,0,21,1,English
4,1,0.168761,0.084875,1250.061078,1383.643005,1891.738561,0.074272,-320.786858,109.852414,-13.965514,...,0,0,0,1,3,0,0,60,1,English


# Perform pandas profiling

In [3]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='../data/music_dataset_output.html')

In [4]:
df_clean = df.drop(columns=[ 'spectral_centroid','spectral_bandwidth'])
from pandas_profiling import ProfileReport
prof = ProfileReport(df_clean)
prof.to_file(output_file='../data/music_dataset_output_clean.html')

## Interpretation 
It has been observed through the profiling that :
1- Most of the audio features contain outliers,
2- Most of the audio features are highly correlated, thus need to remove the most useless features 
3- Data should be normalized 

# Correlation matrix

## All features correlation matrix 

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,track id,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness,mood,liked,disliked,age,gender
track id,1.000000,0.409203,0.337527,0.244838,0.255723,0.252792,0.132064,0.234329,-0.245490,0.097612,0.125133,0.282626,0.297913,0.317500,0.293209,0.266521,0.342521,0.104388,0.186337,0.060331,0.192148,-0.021470,-0.055334,-0.079868,0.095494,-0.001774,0.033470,-0.035889,-0.080450,-0.000833,0.020576,-0.010294,-0.000670,-0.025160,0.060297,0.025009,-0.061634,-0.102205,0.130116,-0.140463,0.056507
chroma_stft,0.409203,1.000000,0.531060,0.424431,0.405789,0.450055,0.246018,0.432346,-0.390328,0.144935,0.400767,0.443211,0.654159,0.557197,0.641426,0.572897,0.579967,0.424013,0.471625,0.249999,0.374220,0.255532,0.311053,0.128499,0.323532,0.164154,0.213572,-0.013145,-0.082410,-0.152894,-0.124624,-0.193690,0.191309,0.138765,0.188961,-0.082432,-0.043053,-0.089351,0.158479,-0.089171,-0.015413
rmse,0.337527,0.531060,1.000000,0.271199,0.357505,0.331667,0.066602,0.747356,-0.259858,0.049216,0.434377,0.418824,0.525942,0.444991,0.565649,0.419593,0.438775,0.197198,0.365635,0.115463,0.182544,0.014174,0.131732,-0.071980,0.194150,-0.011095,0.040058,0.013700,-0.117498,-0.062151,-0.009775,-0.145663,0.117960,0.118952,0.019069,-0.045923,-0.047389,-0.048916,0.106349,-0.083390,0.025714
spectral_centroid,0.244838,0.424431,0.271199,1.000000,0.911651,0.978491,0.861181,0.610184,-0.904885,-0.077204,0.279373,-0.051771,0.301249,0.115881,0.400287,0.251375,0.430317,0.100129,0.298473,-0.036733,0.341765,-0.004375,0.243123,-0.167069,0.166211,-0.073901,0.132426,0.027781,-0.062191,-0.084908,-0.043335,-0.150444,0.103513,0.133302,0.037235,-0.063287,-0.036002,-0.041954,0.081413,-0.038457,0.009980
spectral_bandwidth,0.255723,0.405789,0.357505,0.911651,1.000000,0.951220,0.614921,0.674851,-0.912150,0.060330,0.425859,0.064361,0.397650,0.184505,0.474696,0.265101,0.480054,0.058716,0.300408,-0.102724,0.309867,-0.071978,0.214669,-0.238975,0.170893,-0.163883,0.082988,0.015518,-0.072879,-0.033319,0.000578,-0.128816,0.064454,0.094874,-0.001921,-0.034505,-0.036656,-0.046400,0.090528,-0.043653,0.007882
rolloff,0.252792,0.450055,0.331667,0.978491,0.951220,1.000000,0.764288,0.664892,-0.899173,-0.022880,0.348790,0.017145,0.358173,0.154762,0.440310,0.274727,0.462440,0.113936,0.327795,-0.042935,0.345252,-0.011178,0.244633,-0.198398,0.171832,-0.093904,0.132805,0.023543,-0.064078,-0.075295,-0.032319,-0.157034,0.102680,0.123828,0.032852,-0.058272,-0.036511,-0.045558,0.086158,-0.041192,0.007202
zero_crossing_rate,0.132064,0.246018,0.066602,0.861181,0.614921,0.764288,1.000000,0.407960,-0.687015,-0.225896,0.069402,-0.226115,0.086200,-0.042575,0.184743,0.111746,0.240089,0.058229,0.190850,-0.012692,0.244275,-0.013001,0.171087,-0.092231,0.069343,-0.009851,0.082265,0.046889,-0.037527,-0.098393,-0.061266,-0.124611,0.096880,0.149631,0.035798,-0.070474,-0.022759,-0.017300,0.030843,-0.008760,0.011590
mfcc1,0.234329,0.432346,0.747356,0.610184,0.674851,0.664892,0.407960,1.000000,-0.506740,-0.250030,0.514190,0.143268,0.406300,0.196208,0.495932,0.267573,0.407124,0.033340,0.284828,-0.106288,0.096394,-0.191721,0.109158,-0.285040,0.143716,-0.127141,0.028433,0.047483,-0.102314,-0.072152,-0.012500,-0.190883,0.137075,0.163759,-0.008363,-0.055531,-0.043082,-0.031151,0.076328,-0.048886,0.028478
mfcc2,-0.245490,-0.390328,-0.259858,-0.904885,-0.912150,-0.899173,-0.687015,-0.506740,1.000000,-0.021064,-0.373580,0.047504,-0.352088,-0.153001,-0.449979,-0.287990,-0.447534,-0.120582,-0.309725,0.018681,-0.338531,0.001150,-0.243536,0.172935,-0.163461,0.100045,-0.116025,-0.012785,0.049933,0.061053,0.033028,0.109957,-0.071143,-0.092686,-0.029229,0.044309,0.036267,0.049110,-0.091427,0.047076,0.001264
mfcc3,0.097612,0.144935,0.049216,-0.077204,0.060330,-0.022880,-0.225896,-0.250030,-0.021064,1.00000

## Emotion feature correlation matrix 

In [4]:
emo_feat = [' amazement', ' solemnity', ' tenderness', ' nostalgia', ' calmness',
       ' power', ' joyful_activation', ' tension', ' sadness']
df_emotion = df[emo_feat]
df_emotion.head()

,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness
0,0,0,0,0,0,0,0,0,1
1,0,0,1,1,0,0,0,0,1
2,0,1,0,0,1,0,0,0,1
3,1,1,0,0,1,0,0,0,0
4,0,1,0,1,0,0,0,0,1


In [5]:
emo_corr = df_emotion.corr()
emo_corr.style.background_gradient(cmap='coolwarm')

,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness
amazement,1.000000,-0.034695,0.002308,-0.099185,-0.072947,0.046472,0.155424,-0.114553,-0.146969
solemnity,-0.034695,1.000000,-0.072575,-0.059082,-0.012520,0.012750,-0.098162,-0.040608,-0.034689
tenderness,0.002308,-0.072575,1.000000,0.087216,0.140270,-0.156100,-0.091257,-0.195904,-0.023642
nostalgia,-0.099185,-0.059082,0.087216,1.000000,0.057075,-0.155181,-0.173886,-0.190797,0.118118
calmness,-0.072947,-0.012520,0.140270,0.057075,1.000000,-0.212538,-0.193518,-0.281527,-0.069646
power,0.046472,0.012750,-0.156100,-0.155181,-0.212538,1.000000,0.128937,0.038252,-0.121927
joyful_activation,0.155424,-0.098162,-0.091257,-0.173886,-0.193518,0.128937,1.000000,-0.115181,-0.228275
tension,-0.114553,-0.040608,-0.195904,-0.190797,-0.281527,0.038252,-0.115181,1.000000,0.010221
sadness,-0.146969,-0.034689,-0.023642,0.118118,-0.069646,-0.121927,-0.228275,0.010221,1.000000


## Grouping by "Track id" and by average of the LABELS to release the proportions by people feeling a particular emotion. 

In [7]:
df_emotion = df[emo_feat + ['track id']]
df_emotion.head()

,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness,track id
0,0,0,0,0,0,0,0,0,1,1
1,0,0,1,1,0,0,0,0,1,1
2,0,1,0,0,1,0,0,0,1,1
3,1,1,0,0,1,0,0,0,0,1
4,0,1,0,1,0,0,0,0,1,1


## Grouping Track id by average to get the proportions.

In [9]:
db = df_emotion.groupby('track id')[emo_feat].mean()
db.head()

,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness
track id,,,,,,,,,
1,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.062500,0.312500
2,0.106383,0.170213,0.425532,0.340426,0.744681,0.042553,0.021277,0.021277,0.106383
3,0.173913,0.347826,0.130435,0.173913,0.065217,0.282609,0.413043,0.239130,0.152174
4,0.119048,0.142857,0.523810,0.452381,0.761905,0.023810,0.000000,0.023810,0.142857
5,0.045455,0.295455,0.318182,0.522727,0.318182,0.045455,0.068182,0.181818,0.318182


### Interpretation
For track id = 1 only 14% of people felt amazement, against 33% solemnity, against 20% tenderness, against 29% nostalgia,...
For a complete here is the description of the distribution of the proportions:

In [10]:
db.describe()

,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,sadness
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.134012,0.195961,0.176911,0.254279,0.299257,0.190244,0.261896,0.237685,0.185288
std,0.101423,0.148159,0.169424,0.176113,0.218495,0.185234,0.234975,0.184631,0.157862
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.071429,0.083333,0.037074,0.097727,0.100000,0.045455,0.067803,0.083333,0.072735
50%,0.117647,0.166667,0.133974,0.230769,0.278889,0.142857,0.190154,0.186343,0.151087
75%,0.181818,0.272727,0.285714,0.365875,0.468711,0.282609,0.454545,0.363636,0.285714
max,0.526316,0.700000,0.750000,0.892857,0.916667,1.000000,1.000000,0.928571,0.857143


1- Overall, most of the subjects did not feel any emotion
2- All the subjects do not share the same feelings for the same song,
3- In other words, Trying to predict the emotions only by the means of the audio features is not enough. Unfortunately, we do not possess enough information regarding the user. 